In [30]:
from package.databases.management.longterm import LongTermManagement
from package.databases.management.document import DocumentManagement
from package.databases.session import get_session, Depends

ltm = LongTermManagement()
dm = DocumentManagement()

In [2]:
longterm_id = '1ad51d70-b53c-4ffc-b7b5-d523ead048d6'

In [5]:
import requests

url = "http://localhost:9000/2015-03-31/functions/function/invocations"
payload = {
    # "prompt": "Hi, how are you?",
    "longterm_id": longterm_id
}  # your event payload

response = requests.post(url, json=payload)

print("Status Code:", response.status_code)
print("Response Body:", response.text)


Status Code: 200
Response Body: {"errorMessage": "Unable to locate credentials", "errorType": "NoCredentialsError", "requestId": "a3416fc5-b82e-440d-b435-8e8dd9e30c73", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 15, in lambda_handler\n    response = model.run(system_prompt=system_prompt, messages=messages)\n", "  File \"/var/task/package/llm/ollama.py\", line 67, in run\n    response = model.converse(\n", "  File \"/var/lang/lib/python3.13/site-packages/botocore/client.py\", line 598, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/lang/lib/python3.13/site-packages/botocore/context.py\", line 123, in wrapper\n    return func(*args, **kwargs)\n", "  File \"/var/lang/lib/python3.13/site-packages/botocore/client.py\", line 1043, in _make_api_call\n    http, parsed_response = self._make_request(\n", "  File \"/var/lang/lib/python3.13/site-packages/botocore/client.py\", line 1067, in _make_request\n    return self._endpoint.make_req

In [17]:
import boto3

def get_bedrock_client():
    """
    Create a Bedrock client using the default AWS credentials and region.
    """
    param = {
        "service_name": "bedrock-runtime",
        "region_name": "us-west-2",
        # "aws_access_key_id": None,
        # "aws_secret_access_key": None,
        # "aws_session_token": None,
    }    
    return boto3.client(**param)

def handler(event, context):
    model_name = event.get("model_name", "us.meta.llama3-2-11b-instruct-v1:0")
    prompt = event.get("prompt", "Hello, Bedrock!")
    content = [{"text": prompt}]
    model = get_bedrock_client()
    response = model.converse(
        modelId=model_name,
        messages=[{"role": "user", "content": content}]
    )
    response = response['output']['message']['content'][0]['text']
    return {
        "statusCode": 200,
        "body": "Hello from Bedrock Postgres Lambda!",
        "response": response
    }

In [18]:
handler(event={}, context={})

{'statusCode': 200,
 'body': 'Hello from Bedrock Postgres Lambda!',
 'response': "Yabba dabba doo!  Hello there! I'm Bedrock's friendly AI, here to help and chat with you about all things prehistoric and beyond. What's on your mind? Want to talk about dinosaurs, the Flintstones, or something else entirely?"}

In [10]:
from broai.llm_management.ollama import BedrockOllamaChat

model = BedrockOllamaChat()

In [11]:
model.run("You are a helpful assistant.", messages=[model.UserMessage("Hi, how are you?")])

"I'm doing well, thank you for asking. I'm here to help with any questions or information you might need. How can I assist you today?"

In [21]:
import os
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\Admin\\AppData\\Roaming',
        'APPLICATIONINSIGHTS_CONFIGURATION_CONTENT': '{}',
        'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': '1',
        'APPLICATION_INSIGHTS_NO_STATSBEAT': 'true',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_19700_QTJFHIYPJLXMTWQM',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'DESKTOP-EKNBFHV',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
        'EFC_27848_1262719628': '1',
        'EFC_27848_1592913036': '1',
        'EFC_27848_2283032206': '1',
        'EFC_27848_2775293581': '1',
        'EFC_27848_3789132940': '1',
        'ELECTRON_NO_ATTACH_CONSOLE': '1',
        'ELECTRON_RUN_AS_NODE': '1',
      

In [24]:
import boto3
import json

# Create Lambda client
lambda_client = boto3.client("lambda", region_name="ap-southeast-1")  # Change to your region

# Set your function name and input event
function_name = "test-bedrock-postgres"
payload = {
    "prompt": "What is the capital of France?",
    # "model_name": "anthropic.claude-3-sonnet-20240229-v1:0"
}

# Invoke the function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType="RequestResponse",  # "Event" for async
    Payload=json.dumps(payload),
)

# Read and parse the response
response_payload = json.load(response["Payload"])
print("Lambda Response:", response_payload)


Lambda Response: {'statusCode': 200, 'body': 'Hello from Bedrock Postgres Lambda!', 'response': 'The capital of France is Paris.'}


In [18]:
import boto3
import json

# Create Lambda client
lambda_client = boto3.client("lambda", region_name="ap-southeast-1")  # Change to your region

# Set your function name and input event
function_name = "test-bedrock-postgres-image"
payload = {
    "longterm_id": longterm_id
    # "prompt": "What is the capital of Thailand?",
    # "model_name": "anthropic.claude-3-sonnet-20240229-v1:0"
}

# Invoke the function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType="RequestResponse",  # "Event" for async
    Payload=json.dumps(payload),
)

# Read and parse the response
response_payload = json.load(response["Payload"])
print("Lambda Response:", response_payload)


Lambda Response: {'statusCode': 200, 'body': 'done'}


In [19]:
ltm = LongTermManagement()
# longterm_id
longterm = ltm.read_longterm(longterm_id, session=Depends(get_session))

In [21]:
print(longterm.combo)

**Summary:**

The provided context discusses the task of generating Wikipedia-like articles from scratch, focusing on the pre-writing stage, also known as research. The authors compare their work to existing literature on Wikipedia generation, highlighting the limitations of previous approaches. They emphasize the importance of generating full articles without prior outlines or references, which is a more challenging task.

**Key Points:**

1. The study focuses on generating Wikipedia-like articles from scratch, with an emphasis on the pre-writing stage (research).
2. Existing literature on Wikipedia generation has limitations, such as generating only one paragraph, focusing on a specific domain, or relying on prior outlines or references.
3. The authors' approach aims to generate full articles without prior outlines or references, which is a more challenging task.
4. The study compares their work to prior benchmarks, including Balepur et al. (2023), Qian et al. (2023), Fan and Gardent

In [59]:
document_id = 'fe8d0d2b-5e8f-49ae-922c-026eabedd8d3'
# longterms = ltm.read_longterms_by_document(document_id=document_id, session=Depends(get_session))
longterms = ltm.read_longterms_by_document(document_id=document_id, session=Depends(get_session))

In [64]:
print(longterms[2].id)

1ebafe7c-340c-41b0-869a-a43d9c0c51e4


In [82]:
longterm_ids = [longterm.id for longterm in longterms]

In [83]:
len(longterm_ids)

87

In [84]:
import boto3
import json
import uuid

# Replace this with your actual Step Function ARN
# STATE_MACHINE_ARN = "arn:aws:states:ap-southeast-1:123456789012:stateMachine:broai-arai-enrich-fleet"
STATE_MACHINE_ARN = "arn:aws:states:ap-southeast-1:112557628841:stateMachine:broai-arai-enrich-fleet"

# Create a unique name for this execution
execution_name = f"test-execution-{uuid.uuid4()}"

# Input payload
input_payload = {
    # "longterm_ids": ["id1", "id2"],
    "longterm_ids": longterm_ids[:]
}

# Create Step Functions client
sfn = boto3.client("stepfunctions", region_name="ap-southeast-1")

# Start execution
response = sfn.start_execution(
    stateMachineArn=STATE_MACHINE_ARN,
    name=execution_name,
    input=json.dumps(input_payload)
)

print("Execution started!")
print("Execution ARN:", response["executionArn"])


Execution started!
Execution ARN: arn:aws:states:ap-southeast-1:112557628841:execution:broai-arai-enrich-fleet:test-execution-cba979ed-6afe-4ac3-a80f-b1786317c3e0


In [85]:
import time

execution_arn = response["executionArn"]

while True:
    desc = sfn.describe_execution(executionArn=execution_arn)
    status = desc["status"]
    print(f"Execution status: {status}")
    if status in ("SUCCEEDED", "FAILED", "TIMED_OUT", "ABORTED"):
        break
    time.sleep(2)

if status == "SUCCEEDED":
    output = desc.get("output")
    print("Execution output:", output)
else:
    print(f"Execution ended with status: {status}")


Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: SUCCEEDED
Execution output: {"status":"success","results":[{"Payload":{"processed":"a1b0fdca-848f-47d0-9eee-8f5d9f7d1c66"}},{"Payload":{"processed":"c92d372b-8586-40f5-b6f3-470e607401c2"}},{"Payload":{"processed":"1ebafe7c-340c-41b0-869a-a43d9c0c51e4"}},{"Payload":{"processed":"665713ac-08ee-4d8f-b0bd-b242c51d9348"}},{"Payload":{"processed":"fb138eca-e5e3-4857-bd08-ef313ec5dd4e"}},{"Payload":{"processed":"9c8c726b-c900-4507-a43a-59c53be5a47f"}},{"Payload":{"processed":"cb84687e-24d7-4981-aa40-3c79951636db"}},{"Payload":{"processed":"8859b95b-d13b-49e9-86a3-1402309cd82e"}},{"Payload":{"processed":"303f4cde-2980-44fc-9dbb-987e112556f9"}},{"Payload":{"processed":"8ef45766-f89d-4bec-bb5d-0700bd86f7d3"}},{"Payload":{"processed":"df24f975-43a1-4ad5-9796-3b32f1685aa2"}},{"Payload":{"processed":"9d468142-ae30-456e-9fbb-381

In [86]:
import json
payloads = json.loads(output)['results']
payloads

[{'Payload': {'processed': 'a1b0fdca-848f-47d0-9eee-8f5d9f7d1c66'}},
 {'Payload': {'processed': 'c92d372b-8586-40f5-b6f3-470e607401c2'}},
 {'Payload': {'processed': '1ebafe7c-340c-41b0-869a-a43d9c0c51e4'}},
 {'Payload': {'processed': '665713ac-08ee-4d8f-b0bd-b242c51d9348'}},
 {'Payload': {'processed': 'fb138eca-e5e3-4857-bd08-ef313ec5dd4e'}},
 {'Payload': {'processed': '9c8c726b-c900-4507-a43a-59c53be5a47f'}},
 {'Payload': {'processed': 'cb84687e-24d7-4981-aa40-3c79951636db'}},
 {'Payload': {'processed': '8859b95b-d13b-49e9-86a3-1402309cd82e'}},
 {'Payload': {'processed': '303f4cde-2980-44fc-9dbb-987e112556f9'}},
 {'Payload': {'processed': '8ef45766-f89d-4bec-bb5d-0700bd86f7d3'}},
 {'Payload': {'processed': 'df24f975-43a1-4ad5-9796-3b32f1685aa2'}},
 {'Payload': {'processed': '9d468142-ae30-456e-9fbb-381060adf59d'}},
 {'Payload': {'processed': 'aee5e24c-c076-4c24-9df4-d5ef792ae7bc'}},
 {'Payload': {'processed': '81897d52-c3eb-4569-b306-b488f748ffee'}},
 {'Payload': {'processed': 'c05e50

In [88]:
longterm_id = payloads[-1]['Payload']['processed']
print(ltm.read_longterm(longterm_id, session=Depends(get_session)).combo)

**Summary:**

The context discusses the challenges of using large language models (LLMs) to write grounded, long-form articles, such as full-length Wikipedia pages. Current approaches to generating Wikipedia articles often bypass the pre-writing stage, which involves thorough research and planning. However, this stage is crucial for producing high-quality articles. The authors propose a new approach, called STORM (Synthesis of Topic-based Research and Multi-level Outlining), which aims to automate the pre-writing stage by leveraging external sources and human learning theories. STORM involves two tasks: generating an outline and collecting reference documents, and then using the outline and references to produce the full-length article.

**Key Points:**

1. Current LLMs struggle to write grounded, long-form articles due to limitations in their pre-writing stage.
2. The pre-writing stage involves thorough research and planning, which is essential for producing high-quality articles.
3. 